In [ ]:
import pandas as pd
df_movies_all = pd.read_csv("data_analysis/movies.txt", sep=',')
df_cast_people = pd.read_csv("data_analysis/cast.txt", sep=',')
df_movie_companies = pd.read_csv("data_analysis/companies.txt", sep=',')
df_ratings = pd.read_csv("data_analysis/ratings.txt", sep=',')
df_users = pd.read_csv("data_analysis/users.txt", sep=',')

df_movies_all


In [ ]:
movies_cast_company=pd.read_csv('data_analysis/movies_cast_company.txt')
movies_cast_company

In [ ]:
import random

# Merge the datasets
user_movie_data = df_ratings.merge(movies_cast_company, on="movie_id_ml")
user_data = user_movie_data.merge(df_users, on="user_id")

# Select only numeric columns for aggregation
numeric_user_data = user_data.select_dtypes(include=['number'])

# Perform the aggregation (mean) on the numeric columns only
user_features = numeric_user_data.groupby("user_id").mean()
# Only ratings are aggregated here

# Select a random user
random_user = user_features.sample(n=1)

# Calculate average of non-zero ratings
non_zero_counts = user_features[user_features > 0].count()


non_zero_averages = user_features[user_features > 0].mean()  # Average rating of non-zero values

#print(non_zero_counts)

print(user_data.columns)


In [5]:

import pandas as pd
import numpy as np
from rapidfuzz import fuzz

# Read IMDB title.basics file
imdb_titles = pd.read_csv('data_analysis/title.basics.tsv', sep='\t', low_memory=False, na_values=['\\N'])

def fast_title_match(user_data, imdb_titles):
    # Preprocess IMDB titles
    imdb_titles = imdb_titles.copy()
    
    # Clean and convert columns
    imdb_titles['primaryTitle'] = imdb_titles['primaryTitle'].astype(str)
    imdb_titles['startYear'] = pd.to_numeric(imdb_titles['startYear'], errors='coerce')
    
    # Remove rows with NaN years or titles
    imdb_titles = imdb_titles.dropna(subset=['primaryTitle', 'startYear', 'tconst'])
    
    # Create a mapping dictionary for faster lookup
    title_dict = {
        (row['primaryTitle'].lower(), int(row['startYear'])): row['tconst'] 
        for _, row in imdb_titles.iterrows()
    }
    
    def find_match(title, year):
        # Ensure title and year are in correct format
        if pd.isna(title) or pd.isna(year):
            return None
        
        title = str(title).lower()
        try:
            year = int(year)
        except (ValueError, TypeError):
            return None
        
        # Exact match first
        exact_match = title_dict.get((title, year))
        if exact_match:
            return exact_match
        
        # Fuzzy matching with year range
        for y in range(year-1, year+2):
            candidates = [
                (tconst, fuzz.ratio(title, t.lower())) 
                for (t, yr), tconst in title_dict.items() 
                if yr == y
            ]
            
            if candidates:
                best_match = max(candidates, key=lambda x: x[1])
                return best_match[0] if best_match[1] > 80 else None
        
        return None
    
    # Vectorized matching
    user_data = user_data.copy()
    user_data['tconst'] = user_data.apply(
        lambda row: find_match(row['title'], row['release']), 
        axis=1
    )
    
    return user_data

# Apply the faster method
merged_df_with_tconst = fast_title_match(user_data, imdb_titles)

In [ ]:
# Write to CSV
#merged_df_with_tconst.to_csv('data_analysis/merged_dataset_with_tconst.csv', index=False)

merged_df_with_tconst

In [11]:
import dask.dataframe as dd

# Read with dask
principals = dd.read_csv('data_analysis/title.principals.tsv', 
                          sep='\t', 
                          na_values=['\\N'])

# Filter and compute
directors = principals[principals['category'] == 'director'].compute()
# Merge to get directors
merged_with_directors = merged_df_with_tconst.merge(
    directors[['tconst', 'nconst']], 
    on='tconst', 
    how='left'
)

# Read names to get director names
names = pd.read_csv('data_analysis/name.basics.tsv', sep='\t', low_memory=False, na_values=['\\N'])

# Final merge to get director names
final_df = merged_with_directors.merge(
    names[['nconst', 'primaryName']], 
    on='nconst', 
    how='left'
)

In [ ]:
# Rename column and filter out NaNs
final_df = final_df.rename(columns={'primaryName': 'director'})

# Check total number of rows before filtering
total_rows_before = len(final_df)

# Filter out rows with NaN directors
final_df_with_directors = final_df.dropna(subset=['director'])

# Calculate and print statistics
total_rows_after = len(final_df_with_directors)
nan_count = total_rows_before - total_rows_after
nan_percentage = (nan_count / total_rows_before) * 100

print(f"Total rows before filtering: {total_rows_before}")
print(f"Total rows after filtering out NaN directors: {total_rows_after}")
print(f"Number of rows removed due to NaN directors: {nan_count}")
print(f"Percentage of rows with missing directors: {nan_percentage:.2f}%")

# Optional: Save the filtered DataFrame
#final_df_with_directors.to_csv('final_dataset_with_directors.csv', index=False)

# Preview the first few rows of directors
print("\nSample of directors:")
print(final_df_with_directors[['title', 'director']].head())

In [ ]:
user_data = pd.read_csv("data_analysis/final_dataset.csv")

import random

# Select only numeric columns for aggregation
numeric_user_data = user_data.select_dtypes(include=['number'])

# Perform the aggregation (mean) on the numeric columns only
user_features = numeric_user_data.groupby("user_id").mean()
# Only ratings are aggregated here

# Select a random user
random_user = user_features.sample(n=1)

# Calculate average of non-zero ratings
non_zero_counts = user_features[user_features > 0].count()


non_zero_averages = user_features[user_features > 0].mean()  # Average rating of non-zero values

#print(non_zero_counts)

print(user_data.columns)


In [ ]:
print(user_features)



In [8]:
import re
import ast

# Function to clean and normalize keywords
def clean_keywords(keyword):
    if pd.isnull(keyword):
        return ""
    # Convert to lowercase and remove special characters
    cleaned = keywords.lower()
    cleaned = re.sub(r'[^a-zA-Z0-9, ]', '', cleaned)  # Keep only alphanumerics and commas
    return cleaned

# Function to extract 'cast_name' from the 'cast' column
def extract_cast_names(cast_data):
    try:
        # Convert the stringified list of dictionaries to Python objects
        cast_list = ast.literal_eval(cast_data)
        formatted_names = []
        for entry in cast_list:
            if "cast_name" in entry:
                # Split the name into surname and first name
                cast_name = entry["cast_name"]
                # Split by comma (i.e., "surname, first_name")
                name_parts = cast_name.split(", ")
                if len(name_parts) == 2:
                    surname, first_name = name_parts
                    # Format as "surname first_name"
                    formatted_names.append(f"{surname} {first_name}")
        return ", ".join(formatted_names)
    except (ValueError, SyntaxError):
        return ""

# Function to extract 'name' from the 'company' column
def extract_company_names(company_data):
    try:
        # Convert the stringified list of dictionaries to Python objects
        company_list = ast.literal_eval(company_data)
        # Extract 'name' from each dictionary in the list
        return ", ".join([entry["name"] for entry in company_list if "name" in entry])
    except (ValueError, SyntaxError):
        return ""


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

# Merge the datasets
user_movie_data = df_ratings.merge(movies_cast_company, on="movie_id_ml")
user_data = user_movie_data.merge(df_users, on="user_id")

# Step 1: Clean keywords, cast, and company data
user_data['cleaned_keywords'] = user_data['keyword'].fillna('').str.strip().str.lower()
user_data['cast'] = user_data['cast'].fillna('').str.strip().str.lower()
user_data['company'] = user_data['company'].fillna('').str.strip().str.lower()

user_data['cast_names'] = user_data['cast'].apply(extract_cast_names)
user_data['company_names'] = user_data['company'].apply(extract_company_names)

# Analyze keywords
all_keywords = user_data['cleaned_keywords'].str.split(',').explode()
keyword_counts = Counter(all_keywords)
top_keywords = keyword_counts.most_common(10)
print("Top Keywords:", top_keywords)

# Analyze cast
all_cast = user_data['cast_names'].str.split(',').explode()
cast_counts = Counter(all_cast)
top_cast = cast_counts.most_common(10)
print("Top Cast:", top_cast)

# Analyze companies
all_comp = user_data['company_names'].str.split(',').explode()
comp_counts = Counter(all_comp)
top_comp = comp_counts.most_common(10)
print("Top Companies:", top_comp)

# Step 2: Use CountVectorizer with top keywords, cast, and company names for encoding
top_k = 30  # Adjust based on memory constraints
most_common_keywords = [keyword for keyword, _ in keyword_counts.most_common(top_k)]
most_common_cast = [cast for cast, _ in cast_counts.most_common(top_k)]
most_common_companies = [company for company, _ in comp_counts.most_common(top_k)]

# Vectorizing columns with CountVectorizer
vectorizer_keywords = CountVectorizer(tokenizer=lambda x: x.split(','), lowercase=True, vocabulary=most_common_keywords)
vectorizer_cast = CountVectorizer(tokenizer=lambda x: x.split(','), lowercase=True, vocabulary=most_common_cast)
vectorizer_companies = CountVectorizer(tokenizer=lambda x: x.split(','), lowercase=True, vocabulary=most_common_companies)

# Transform the columns
keyword_matrix = vectorizer_keywords.fit_transform(user_data['cleaned_keywords'])
cast_matrix = vectorizer_cast.fit_transform(user_data['cast_names'])
company_matrix = vectorizer_companies.fit_transform(user_data['company_names'])

# Convert sparse matrices to DataFrames
keyword_df = pd.DataFrame(keyword_matrix.toarray(), columns=vectorizer_keywords.get_feature_names_out())
cast_df = pd.DataFrame(cast_matrix.toarray(), columns=vectorizer_cast.get_feature_names_out())
company_df = pd.DataFrame(company_matrix.toarray(), columns=vectorizer_companies.get_feature_names_out())

# Concatenate encoded data with user_data
user_data = pd.concat([user_data.reset_index(drop=True), keyword_df, cast_df, company_df], axis=1)

# Remove duplicate columns
user_data = user_data.loc[:, ~user_data.columns.duplicated()]

# Step 3: Handle numeric columns and scaling
numeric_user_data = user_data.select_dtypes(include=['number'])
scaler = MinMaxScaler()
numeric_user_data_scaled = pd.DataFrame(
    scaler.fit_transform(numeric_user_data),
    columns=numeric_user_data.columns
)

# Aggregate user data by mean
user_features = numeric_user_data.groupby("user_id").mean()

# Check correlation
corr = user_features.corr()
print(corr["rating"].sort_values(ascending=False))

# Step 4: Prepare data for model training
X = user_features.drop("rating", axis=1)  # Drop target column
y = user_features["rating"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importance scores
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# Drop features with zero importance
non_zero_features = feature_importances[feature_importances > 0].index
X = X[non_zero_features]
X_train = X_train[non_zero_features]
X_test = X_test[non_zero_features]

# Retrain the model with reduced features
rf.fit(X_train, y_train)

# Plot the top_n most important features
top_n = 100
top_features = feature_importances.nlargest(top_n)

plt.figure(figsize=(15, 8))
top_features.plot(kind="bar")
plt.title(f"Top {top_n} Feature Importances")
plt.ylabel("Importance Score")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Optionally, print the most important features
print("Most important features:")
print(top_features)


In [ ]:
user_data.columns


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Calculate the number of non-zero entries for each feature
feature_non_zero_count = (user_data != 0).sum(axis=0)

# Print out the number of rows in which each feature is found
print("Number of non-zero rows for each feature:")
print(feature_non_zero_count)

# Step 2: Plot a histogram of the number of rows each feature is found in
plt.figure(figsize=(10, 6))
plt.hist(feature_non_zero_count, bins=50, edgecolor='black')
plt.title('Histogram of the Number of Rows in Which Each Feature is Found')
plt.xlabel('Number of Non-Zero Rows per Feature')
plt.ylabel('Frequency')
plt.show()



In [ ]:
#log scaling
feature_non_zero_count = (user_data != 0).sum(axis=0)
plt.figure(figsize=(10, 6))
plt.hist(feature_non_zero_count, bins=50, edgecolor='black', log=True)
plt.title('Histogram of Non-Zero Feature Counts')
plt.xlabel('Number of Non-Zero Rows per Feature')
plt.ylabel('Log Frequency')
plt.tight_layout()
plt.show()


In [ ]:
datasets = {
    "user_data": user_data,
    "movies_cast_company": movies_cast_company,
    "df_movies_all": df_movies_all,
    "df_cast_people": df_cast_people,
    "df_movie_companies": df_movie_companies,
    "df_ratings": df_ratings,
    "df_users": df_users,
}
for df_name, df in datasets.items():
    print(f"Dataset: {df_name}")
    print(df.info())
    print(df.describe())
    print("\n")


In [ ]:
adding director

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Load your dataset (ensure it has 'title' and 'imdb_id' columns)
data_path = 'data_analysis/movies_cast_company.txt'  # Replace with your dataset file path
links = pd.read_csv('data_analysis/links.csv')  # MovieLens links dataset

# Merge to map movie_id_ml to IMDb IDs
movies_with_imdb = movies.merge(links, left_on='movie_id_ml', right_on='movieId', how='left')

# Create the IMDb ID string required for IMDb URLs
movies_with_imdb['imdb_id'] = movies_with_imdb['imdbId'].apply(lambda x: f"tt{int(x):07d}" if not pd.isna(x) else None)

# Save the updated dataset
movies_with_imdb.to_csv('movies_with_imdb_ids.csv', index=False)
print("Saved dataset with IMDb IDs as 'movies_with_imdb_ids.csv'")

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def fetch_director_from_imdb(imdb_id):
    """
    Fetch the director's name for a movie using its IMDb ID.
    """
    url = f"https://www.imdb.com/title/{imdb_id}/"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch IMDb page for ID {imdb_id}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the director(s) in the IMDb page
    director_section = soup.find_all('div', {'class': 'credit_summary_item'})
    directors = []
    
    for section in director_section:
        if 'Director' in section.get_text():
            director_links = section.find_all('a')
            directors = [a.text.strip() for a in director_links]
            break
    
    if directors:
        return ', '.join(directors)
    
    return None  # Director not found

# Load dataset and merge with IMDb IDs
movies = pd.read_csv('movies_with_imdb_ids.csv')  # Dataset with IMDb IDs

# Filter out movies without IMDb IDs
movies = movies[movies['imdb_id'].notna()]

# Fetch directors using the updated scraping function
directors = []
for index, row in movies.iterrows():
    imdb_id = row['imdb_id']
    print(f"Fetching director for IMDb ID: {imdb_id} ({row['title']})")
    director = fetch_director_from_imdb(imdb_id)
    directors.append(director)
    time.sleep(2)  # Increase delay to reduce rate limiting

movies['director'] = directors

# Save the updated dataset
movies.to_csv('movies_with_directors.csv', index=False)
print("Directors added and dataset saved to 'movies_with_directors.csv'")
